## Imports

In [2]:
from __future__ import print_function
import keras
import os
from keras.datasets import mnist
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

Using TensorFlow backend.


## Parameters

In [3]:
batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

## Load Data

In [4]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


## Train Model

In [5]:
model_path = "./model_weights.h5"
if os.path.exists(model_path):
    model = load_model(model_path)
    print("Loaded model...")
else:
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=input_shape))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adadelta(),
                  metrics=['accuracy'])

    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test))
    
    model.save(model_path)
    print("Saved model...")

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Loaded model...
Test loss: 0.02600341277925363
Test accuracy: 0.9918000102043152


## Train Model (Split)

In [7]:
x_train_seperated = {}
for target_label in range(10):
    x_train_seperated[target_label] = []
    for i, label in enumerate(y_train):
        if list(label).index(1) == target_label:
            x_train_seperated[target_label].append(x_train[i])
            
x_test_seperated = {}
for target_label in range(10):
    x_test_seperated[target_label] = []
    for i, label in enumerate(y_test):
        if list(label).index(1) == target_label:
            x_test_seperated[target_label].append(x_test[i])

In [9]:
model_split = load_model(model_path)

# train model on every class, iterately (one client per class)
for label in range(10):
    print('Training on client {} using class {}'.format(label, label))
    model.fit(x_train_separated[label], y_train)
    

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

## Train Model (Label Poisoning)

In [6]:
model_path = "./model_weights.h5"
adv_model = load_model(model_path)
print("Loaded model...")

adv_model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

adv_model_path = "./adv_model_weights.h5"
adv_model.save(adv_model_path)
print("Saved model...")

score = adv_model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Loaded model...
Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 5s 82us/step - loss: 0.0239 - accuracy: 0.9924 - val_loss: 0.0235 - val_accuracy: 0.9932
Epoch 2/12
23168/60000 [==========>...................] - ETA: 2s - loss: 0.0215 - accuracy: 0.9921

KeyboardInterrupt: 